# Design SARS-CoV-2 spike RBD mutants
This notebook chooses/designs spikes with mutant RBDs.

## Setup
Import Python modules:

In [1]:
import collections
import itertools
import json
import os
import subprocess
import tempfile
import urllib

import altair as alt

import Bio.Entrez
import Bio.SeqIO

import numpy

import pandas as pd

import ruamel.yaml as yaml

Read configuration:

In [2]:
with open("config.yaml") as f:
    config = yaml.YAML().load(f)

## Get the reference RBD

In [3]:
Bio.Entrez.email = "example@example.com"

print(f"Getting reference spike from accession {config['ref_spike']}")
with Bio.Entrez.efetch(id=config["ref_spike"], rettype="gb", retmode="text", db="protein") as f:
    ref_spike = Bio.SeqIO.read(f, "gb")
print(f"Got spike of length {len(ref_spike)}")

rbd_coords = config["rbd_coords"]

ref_rbd = {r: ref_spike[r - 1] for r in range(rbd_coords[0], rbd_coords[1] + 1)}

Getting reference spike from accession YP_009724390
Got spike of length 1273


## Get RBD mutations in each Pango lineage

In [4]:
pango_json = config["pango_json"]
print(f"Reading Pango clade definitions from {pango_json}")
with urllib.request.urlopen(pango_json) as url:
    pango_lineages = json.load(url)
print(f"Read definitions for {len(pango_lineages)} lineages")


def parse_spike_rbd_muts(lineage_d):
    """Parse spike RBD mutations from dict for a lineage."""
    rbd_muts = []
    for mut in lineage_d["aaSubstitutions"] + lineage_d["aaDeletions"]:
        if mut.startswith("S:"):
            mut = mut.split(":")[1]
            wt = mut[0]
            r = int(mut[1: -1])
            m = mut[-1]
            if rbd_coords[0] <= r <= rbd_coords[1]:
                assert wt == ref_rbd[r]
                rbd_muts.append((wt, r, m))
    return rbd_muts
            
pango_rbd_muts = {pango: parse_spike_rbd_muts(d) for (pango, d) in pango_lineages.items()}

Reading Pango clade definitions from https://raw.githubusercontent.com/corneliusroemer/pango-sequences/main/data/pango-consensus-sequences_summary.json
Read definitions for 3916 lineages


## Get and import the escape calculator

In [5]:
# get and import the module
_ = urllib.request.urlretrieve(
    config["escape_calculator_module_url"],
    "escapecalculator.py",
)

import escapecalculator

## Now make designs for each parent

In [6]:
repetition_downweight = config["repetition_downweight"]
allow_reversions_to_ref = config["allow_reversions_to_ref"]
categories = config["categories"]

all_mutants = []
for (parent, parent_d), category in itertools.product(config["parent_specs"].items(), categories):
    nmutants = parent_d["nmutants"]
    nmutations = categories[category]
    with open(parent_d["specs"]) as f:
        parent_config = yaml.YAML().load(f)

    # make a data frame that includes parent and ref amino acid at each site
    parent_aas = (
        # first get reference and parent amino acid for each site
        pd.Series(ref_rbd).rename_axis("site").rename("ref_amino_acid").reset_index()
        .merge(
            pd.DataFrame(
                pango_rbd_muts[parent],
                columns=["ref_amino_acid", "site", "parent_amino_acid"],
            ),
            how="outer",
            on=["ref_amino_acid", "site"],
        )
        .assign(
            parent_amino_acid=lambda x: x["parent_amino_acid"].where(
                x["parent_amino_acid"].notnull(), x["ref_amino_acid"])
        )
    )
    assert len(parent_aas) == parent_aas["site"].nunique() == len(ref_rbd)

    parent_muts_from_ref = (
        parent_aas
        .query("ref_amino_acid != parent_amino_acid")
        .set_index("site")
        ["parent_amino_acid"]
        .to_dict()
    )

    # set up escape calculator
    escape_calc = escapecalculator.EscapeCalculator(virus=parent_config["escapecalculator"]["virus"])
    # get sites that differ between parent and virus used to initialize escape calculator
    parent_escape_calc_virus_diff_sites = list(set([
        tup[1]
        for tup in set(pango_rbd_muts[parent]).symmetric_difference(
            pango_rbd_muts[parent_config["escapecalculator"]["virus"]]
        )
    ]))

    # get RBD deep mutational scanning data
    rbd_dms = (
        pd.read_csv(parent_config["rbd_dms"]["data"])
        .query("target == @parent_config['rbd_dms']['target']")
        .rename(columns={"position": "site", "mutant": "amino_acid"})
        [["site", "amino_acid", "delta_bind", "delta_expr"]]
        .assign(
            delta_bind=lambda x: x["delta_bind"].clip(upper=parent_config["rbd_dms"]["clip"]),
            delta_expr=lambda x: x["delta_expr"].clip(upper=parent_config["rbd_dms"]["clip"]),
        )
    )
    # normalize values to parent amino acids
    rbd_dms = (
        rbd_dms
        .merge(parent_aas[["site", "parent_amino_acid"]], on="site", validate="many_to_one")
        .query("parent_amino_acid == amino_acid")
        .rename(columns={"delta_expr": "parent_expr", "delta_bind": "parent_bind"})
        .drop(columns=["amino_acid", "parent_amino_acid"])
        .merge(rbd_dms, on="site", validate="one_to_many")
        .assign(
            delta_bind=lambda x: x["delta_bind"] - x["parent_bind"],
            delta_expr=lambda x: x["delta_expr"] - x["parent_expr"],
        )
        .drop(columns=["parent_expr", "parent_bind"])
        .rename(columns={"delta_bind": "rbd_delta_bind", "delta_expr": "rbd_delta_expr"})
    )
    assert len(rbd_dms)

    # get all-clade fitness estimates 
    fitness = (
        pd.read_csv(parent_config["fitness_estimates"]["fitness"])
        .query("gene == 'S'")
        .query("expected_count >= @parent_config['fitness_estimates']['fitness_min_count']")
        .rename(columns={"aa_site": "site", "aa": "amino_acid"})
        .assign(fitness=lambda x: x["fitness"].clip(upper=parent_config["fitness_estimates"]["clip"]))
        [["site", "amino_acid", "fitness"]]
    )
    assert len(fitness)

    # get clade fitness effects
    by_clade_fitness = (
        pd.read_csv(parent_config["fitness_estimates"]["by_clade"])
        .query("gene == 'S'")
        .query("clade == @parent_config['fitness_estimates']['clade']")
        .query(
            "(expected_count >= @parent_config['fitness_estimates']['clade_min_count'])"
            "or (actual_count >= @parent_config['fitness_estimates']['clade_min_count'])"
        )
        .rename(columns={"aa_site": "site", "mutant_aa": "amino_acid", "delta_fitness": "clade_fitness"})
        .assign(clade_fitness=lambda x: x["clade_fitness"].clip(upper=parent_config["fitness_estimates"]["clip"]))
        [["site", "amino_acid", "clade_fitness"]]
    )
    assert len(by_clade_fitness)

    # get spike DMS
    spike_dms = (
        pd.read_csv(parent_config["spike_dms"]["csv"])
        .rename(columns={"mutant": "amino_acid"})
        [["site", "amino_acid", "human sera escape", "spike mediated entry", "ACE2 binding"]]
    )
    # normalize values to parent amino acids
    spike_dms = (
        spike_dms
        .merge(parent_aas[["site", "parent_amino_acid"]], on="site", validate="many_to_one")
        .query("parent_amino_acid == amino_acid")
        .rename(
            columns={
                "human sera escape": "parent_human sera escape",
                "spike mediated entry": "parent_spike mediated entry",
                "ACE2 binding": "parent_ACE2 binding",
            }
        )
        .drop(columns=["amino_acid", "parent_amino_acid"])
        .merge(spike_dms, on="site", validate="one_to_many")
        .assign(
            spike_escape=lambda x: x["human sera escape"] - x["parent_human sera escape"],
            spike_entry=lambda x: x["spike mediated entry"] - x["parent_spike mediated entry"],
            spike_ACE2_binding=lambda x: x["ACE2 binding"] - x["parent_ACE2 binding"],
        )
        [["site", "amino_acid", "spike_escape", "spike_entry", "spike_ACE2_binding"]]
    )
    assert len(spike_dms)

    # now add each phenotype and its weight
    parent_phenotypes = (
        parent_aas
        # add fitness estimates, only keeping mutations with estimates
        .merge(fitness, on="site", validate="one_to_many", how="inner")
        .assign(fitness_weight=parent_config["fitness_estimates"]["weights"]["fitness"])
        # add clade fitness estimates, clipping lower at zero and also setting missing to zero
        .merge(by_clade_fitness, on=["site", "amino_acid"], validate="one_to_one", how="left")
        .assign(
            clade_fitness=lambda x: x["clade_fitness"].where(x["clade_fitness"] > 0, 0),
            clade_fitness_weight=parent_config["fitness_estimates"]["weights"]["by_clade_effect"],
        )
        # add RBD DMS, only keeping mutations with measurements
        .merge(rbd_dms, on=["site", "amino_acid"], validate="one_to_one", how="inner")
        .assign(
            rbd_delta_bind_weight=parent_config["rbd_dms"]["weights"]["delta_bind"],
            rbd_delta_expr_weight=parent_config["rbd_dms"]["weights"]["delta_expr"],
        )
        # add spike DMS, only keeping mutations with measurements
        .merge(spike_dms, on=["site", "amino_acid"], validate="one_to_one", how="inner")
        .assign(
            spike_escape_weight=parent_config["spike_dms"]["weights"]["sera escape"],
            spike_entry_weight=parent_config["spike_dms"]["weights"]["spike mediated entry"],
            spike_ACE2_binding_weight=parent_config["spike_dms"]["weights"]["ACE2 binding"],
        )
    )

    # Design each mutant
    mut_counts = collections.defaultdict(int)  # count how many times each mutation already in a mutant
    for imutant in range(nmutants):
        mutant_name = f"{parent}-{category}-mutant-{imutant + 1}"
        print(f"\nDesigning {mutant_name}")
        
        mutated_sites = []  # sites mutated in this variant
        mutated_df = []
        mutations = []
        muts_from_ref = parent_muts_from_ref.copy()
        for imutation in range(nmutations):
            # we re-do escape calculator for each new mutated sites
            mutant_phenotypes = parent_phenotypes.merge(
                escape_calc.escape_per_site(parent_escape_calc_virus_diff_sites + mutated_sites)
                .rename(columns={"retained_escape": "escape_calc"})
                [["site", "escape_calc"]]
                .assign(escape_calc_weight=parent_config["escapecalculator"]["weight"]),
                on="site",
                validate="many_to_one",
            )

            # ignore parent to parent mutations
            mutant_phenotypes = mutant_phenotypes.query("parent_amino_acid != amino_acid")

            # potentially ignore mutations to reference
            if not config["allow_reversions_to_ref"]:
                mutant_phenotypes = mutant_phenotypes.query("ref_amino_acid != amino_acid")

            # get the phenotypes
            phenos = [
                c for c in mutant_phenotypes.columns
                if not c.endswith("_weight") and c not in {"site", "amino_acid", "parent_amino_acid", "ref_amino_acid"}
            ]
            assert all(f"{p}_weight" in set(mutant_phenotypes.columns) for p in phenos)

            # assign score to each mutation
            mutant_phenotypes["score"] = mutant_phenotypes["escape_calc"]**mutant_phenotypes["escape_calc_weight"]
            for pheno in phenos:
                if pheno != "escape_calc":
                    mutant_phenotypes["score"] = mutant_phenotypes["score"] * numpy.exp(
                        mutant_phenotypes[pheno] * mutant_phenotypes[f"{pheno}_weight"]
                    )

            # re-weight mutations already added to variants
            mutant_phenotypes = (
                mutant_phenotypes
                .assign(
                    mutation=lambda x: x["parent_amino_acid"] + x["site"].astype(str) + x["amino_acid"],
                    reweight=lambda x: config["repetition_downweight"]**x["mutation"].map(mut_counts),
                    score=lambda x: x["score"] * x["reweight"],
                )
            )

            # get top scoring mutation not in already mutated sites
            mutation_row = (
                mutant_phenotypes
                .query("site not in @mutated_sites")
                .query("score.notnull()")
                .sort_values("score")
                [["site", "mutation", "ref_amino_acid", "parent_amino_acid", "amino_acid", *phenos]]
                .tail(1)
            )
            site = mutation_row["site"].iloc[0]
            parent_amino_acid = mutation_row["parent_amino_acid"].iloc[0]
            amino_acid = mutation_row["amino_acid"].iloc[0]
            mutation = mutation_row["mutation"].iloc[0]
            mutated_sites.append(site)
            mutated_df.append(mutation_row)
            mutations.append(mutation)
            mut_counts[mutation] += 1
            muts_from_ref[site] = amino_acid

        muts_from_ref = " ".join([f"{ref_rbd[site]}{site}{mut}" for (site, mut) in sorted(muts_from_ref.items())])
        mutations = " ".join(mutations)
        all_mutants.append((mutant_name, parent, mutations, muts_from_ref))
        mutated_df = pd.concat(mutated_df, ignore_index=True)
        print(f"Contains the following mutations relative to {parent}: {mutations}")
        display(mutated_df)

all_mutants = pd.DataFrame(
    all_mutants,
    columns=["name", "parent_strain", "mutations_from_parent", "mutations_from_reference"],
)


Designing KP.2-moderate-mutant-1
Contains the following mutations relative to KP.2: K444T A348P F490S K440R I468L


,site,mutation,ref_amino_acid,parent_amino_acid,amino_acid,fitness,clade_fitness,rbd_delta_bind,rbd_delta_expr,spike_escape,spike_entry,spike_ACE2_binding,escape_calc
0,444,K444T,K,K,T,1.82650,2.0000,-0.10,-0.19,0.09751,0.12810,-0.34620,0.042782
1,348,A348P,A,A,P,0.26923,0.0000,0.15,0.25,0.22140,0.02453,-0.21020,0.051010
2,490,F490S,F,F,S,1.78160,1.7269,-0.03,0.14,0.19840,0.03862,-0.35110,0.022215
3,440,K440R,N,K,R,2.00000,1.3768,-0.10,-0.18,-0.02675,-0.07078,-0.23790,0.035212
4,468,I468L,I,I,L,-0.11917,0.0000,0.03,-0.05,-0.05109,-0.01950,0.08824,0.048523



Designing KP.2-moderate-mutant-2
Contains the following mutations relative to KP.2: K444R A348S F490V I468M A475S


,site,mutation,ref_amino_acid,parent_amino_acid,amino_acid,fitness,clade_fitness,rbd_delta_bind,rbd_delta_expr,spike_escape,spike_entry,spike_ACE2_binding,escape_calc
0,444,K444R,K,K,R,2.00000,1.9911,-0.10,-0.04,-0.021040,-0.051940,0.04488,0.042782
1,348,A348S,A,A,S,0.22659,0.0000,-0.12,-0.21,0.200300,0.005438,-0.16630,0.051010
2,490,F490V,F,F,V,2.00000,0.0000,0.06,-0.06,0.332300,-0.087350,-0.42020,0.022215
3,468,I468M,I,I,M,-0.80286,0.0000,-0.05,-0.02,0.006988,-0.001435,0.09491,0.049475
4,475,A475S,A,A,S,-0.74560,1.3679,-0.40,-0.09,0.184800,-0.082650,-0.48310,0.032735



Designing KP.2-moderate-mutant-3
Contains the following mutations relative to KP.2: K444M F490Y A348P I468V K440R


...

,site,mutation,ref_amino_acid,parent_amino_acid,amino_acid,fitness,clade_fitness,rbd_delta_bind,rbd_delta_expr,spike_escape,spike_entry,spike_ACE2_binding,escape_calc
0,440,K440T,N,K,T,0.954800,0.00000,-0.35,0.11,0.12730,0.00435,-0.90900,0.140756
1,356,K356T,K,K,T,2.000000,2.00000,-0.06,0.05,0.36490,-0.13620,-0.37850,0.041429
2,490,S490Y,F,S,Y,1.231500,0.73112,0.23,0.05,0.07370,0.03142,0.26923,0.029426
3,444,K444R,K,K,R,2.000000,0.94358,-0.09,-0.05,-0.02104,-0.05194,0.04488,0.027686
4,405,N405G,D,N,G,-0.023952,0.00000,0.35,0.06,-0.50860,-0.08596,1.92300,0.045611



Designing HK.3.2-aggressive-mutant-1
Contains the following mutations relative to HK.3.2: K440R K356T K444T S494P N405Y K462T L452R T500A


,site,mutation,ref_amino_acid,parent_amino_acid,amino_acid,fitness,clade_fitness,rbd_delta_bind,rbd_delta_expr,spike_escape,spike_entry,spike_ACE2_binding,escape_calc
0,440,K440R,N,K,R,2.00000,0.41552,-0.07,0.06,-0.02675,-0.07078,-0.23790,0.140756
1,356,K356T,K,K,T,2.00000,2.00000,-0.06,0.05,0.36490,-0.13620,-0.37850,0.041429
2,444,K444T,K,K,T,1.82650,2.00000,-0.18,0.19,0.09751,0.12810,-0.34620,0.034671
3,494,S494P,S,S,P,2.00000,2.00000,0.26,0.07,0.12060,0.06785,0.26350,0.016804
4,405,N405Y,D,N,Y,-0.96129,0.00000,0.19,-0.17,0.21140,0.05109,-0.15830,0.045412
5,462,K462T,K,K,T,-0.69724,0.00000,-0.03,-0.09,0.30540,-0.02060,-0.43700,0.033917
6,452,L452R,L,L,R,0.00000,2.00000,0.17,-0.01,0.18160,0.12860,0.15710,0.012394
7,500,T500A,T,T,A,0.14122,1.48470,-0.19,-0.09,0.05544,0.03091,0.00796,0.019028



Designing HK.3.2-aggressive-mutant-2
Contains the following mutations relative to HK.3.2: K440T K356T S490Y K444R N405G K462R A348S K478R


,site,mutation,ref_amino_acid,parent_amino_acid,amino_acid,fitness,clade_fitness,rbd_delta_bind,rbd_delta_expr,spike_escape,spike_entry,spike_ACE2_binding,escape_calc
0,440,K440T,N,K,T,0.954800,0.00000,-0.35,0.11,0.12730,0.004350,-0.90900,0.140756
1,356,K356T,K,K,T,2.000000,2.00000,-0.06,0.05,0.36490,-0.136200,-0.37850,0.041429
2,490,S490Y,F,S,Y,1.231500,0.73112,0.23,0.05,0.07370,0.031420,0.26923,0.029426
3,444,K444R,K,K,R,2.000000,0.94358,-0.09,-0.05,-0.02104,-0.051940,0.04488,0.027686
4,405,N405G,D,N,G,-0.023952,0.00000,0.35,0.06,-0.50860,-0.085960,1.92300,0.045611
5,462,K462R,K,K,R,-1.093400,0.00000,0.00,-0.09,0.25190,0.061530,-0.61890,0.033644
6,348,A348S,A,A,S,0.226590,1.20000,-0.03,0.26,0.20030,0.005438,-0.16630,0.013186
7,478,K478R,T,K,R,2.000000,2.00000,-0.24,-0.02,0.02994,0.037840,-0.09573,0.009716


Write designed mutants to file:

In [7]:
print(f"Writing designed mutants to {config['designed_mutants']}")
all_mutants.to_csv(config["designed_mutants"], index=False)

Writing designed mutants to designed_mutants.csv
